<img src="assets/header_notebook.jpg" />

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Librairies</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

Run the following commands on terminal to install PySR:


```
python
import pysr
pysr.install(precompile = False)
```


Now you are good to go !


In [ ]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split

np.random.seed(0)
X = 2 * np.random.randn(100, 5)
y = 2.5382 * np.cos(X[:, 3]) + X[:, 0] ** 2 - 2

default_pysr_params = dict(
    populations=30,
    model_selection="best",
)

model = PySRRegressor(
    niterations=30,
    binary_operators=["plus", "mult"],
    unary_operators=["cos", "exp", "sin"],
    **default_pysr_params
)

model.fit(X, y)

/home/vmangeleer/anaconda3/envs/TFE/lib/python3.9/site-packages/pysr/sr.py:1296: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Compiling Julia backend...


In [ ]:
import pysr
import numpy as     np

from   pysr        import PySRRegressor
from   matplotlib  import pyplot as plt
from   sympy       import *
from   julia       import Julia

# Installing Julia
julia = Julia(compiled_modules = False, threads = 'auto')

from julia       import Main
from julia.tools import redirect_output_streams

# Used to display correctly Julia's output
redirect_output_streams()

# Instaling correctly PySR
pysr.install(precompile = False)

In [ ]:
!python3 -m pysr install

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>PySR - Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
np.random.seed(0)

# Initialization of the fake datasets
u =  np.random.randn(1, 64, 64)
v =  np.random.randn(1, 64, 64)
q =  np.random.randn(1, 64, 64)

print(u.shape, v.shape, q.shape)

# ------- SHAPING X0 --------
# Note : 1st dimension = Values for u, v, q
#        2nd dimension = snapshot index
#        3rd dimension = input variable index
u = u.flatten('F')
v = v.flatten('F')
q = q.flatten('F')

print(u.shape, v.shape, q.shape)

u = np.expand_dims(u, axis = 1)
v = np.expand_dims(v, axis = 1)
q = np.expand_dims(q, axis = 1)

print(u.shape, v.shape, q.shape)

X0 = np.concatenate((u, v, q), axis = 1)

print(X0.shape)

X0 = np.expand_dims(X0, axis = 1)

print(X0.shape)

# ------- SHAPING X1 --------
# X1 = cos(u) + v * q^2

X1 = np.cos(X0[:, 0, 0]) + X0[:, 0, 1] * X0[:, 0, 2]**2

print(X1.shape)

X1 = np.expand_dims(X1, axis = (1, 2))

print(X1.shape)

# ------- SHAPING X2 --------
# X2 = [X0, X1]

X2 = np.concatenate((X0, X1), axis = 2)

print(X2.shape)

# ------- SHAPING X3 --------
# X3 = exp(u) + q * X1 - v**2

X3 = np.exp(X2[:, 0, 0]) - X2[:, 0, 1]**2 + X2[:, 0, 2] * X2[:, 0, 3]

print(X3.shape)

X3 = np.expand_dims(X3, axis = (1, 2))

print(X3.shape)

In [ ]:
"""
---- Equations ----
X0 = 1st dimension = Values for u, v, q
     2nd dimension = snapshot index
     3rd dimension = input variable index
X1 = cos(u) + v * q^2
X2 = [X0, X1]
X3 = exp(u) + q * X1 - v**2
"""

# Random indexes (small sample size needed by PySR)
idx = np.random.randint(0, 4095, size = 1000)

# Initialization of the models
model_f = PySRRegressor(
    model_selection  = 'best',
    binary_operators = ["plus", "sub", "mult", "div"],
    unary_operators  = ["square", "cube", "exp", "log", "sqrt", "sin", "cos", "tanh", "erf"],
    niterations      = 2,
    procs            = 4,
    populations      = 100,
)

model_g = PySRRegressor(
    model_selection  = 'best',
    binary_operators = ["plus", "sub", "mult", "div"],
    unary_operators  = ["square", "cube", "exp", "log", "sqrt", "sin", "cos", "tanh", "erf"],
    niterations      = 2,
    procs            = 4,
    populations      = 100,
)

In [ ]:
"""
g(u, v, q, f(X0)) = exp(u) + q * X1 - v**2
"""

print(X0.shape, X1.shape)

X0_PYSR = X0.squeeze(axis = 1)
X1_PYSR = X1.squeeze(axis = 1)

print(X0_PYSR.shape, X1_PYSR.shape)

# Fitting the model (1) - Finding function f
model_f.fit(X0_PYSR[idx, :], X1_PYSR[idx, :])

In [ ]:
# ---- Table of results ----
model_f

In [ ]:
# ---- Best equation ----
model_f.sympy()

In [ ]:
# ---- Plotting equation ----
from matplotlib import pyplot as plt

plt.scatter(X1_PYSR[:, :], model_f.predict(X0_PYSR[:, :]))
plt.xlabel('Truth')
plt.ylabel('Prediction')
plt.show()

In [ ]:
"""
g(u, v, q, f(X0)) = exp(u) + q * X1 - v**2
"""

print(X2.shape, X3.shape)

X2_PYSR = X2.squeeze(axis = 1)
X3_PYSR = X3.squeeze(axis = 1)

print(X2_PYSR.shape, X3_PYSR.shape)

# Fitting the model (2) - Finding function g
model_g.fit(X2_PYSR[idx, :], X3_PYSR[idx, :])

In [ ]:
# ---- Table of results ----
model_g

In [ ]:
# ---- Best equation ----
model_g.sympy()

In [ ]:
# ---- Plotting equation ----
from matplotlib import pyplot as plt

plt.scatter(X3_PYSR[:, :], model_g.predict(X2_PYSR[:, :]))
plt.xlabel('Truth')
plt.ylabel('Prediction')
plt.show()

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Sympy - Playground & Notes</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# Convert the model to a pytorch model ! (Can use torch tensors as inputs)
torch_model = model_f.pytorch()

In [ ]:
# Convert the model to a sympy model ! (Can be used to vizualize expression after taking divergence)
sympy_model = model_f.sympy()

# Defining own coordinate system
base = x0, x1, x2 = symbols("x0 x1 x2")

# ----- Functions -------
from sympy import *

def gradient(f, coords):
  return Matrix([diff(f, c) for c in coords])

def divergence(f, coords):
  terms = [diff(f, c) for c in coords]
  return Add(*terms)

def curl(f, coords):
  return Matrix([
      diff(f, coords[1]) - diff(f, coords[2]),
      diff(f, coords[2]) - diff(f, coords[0]),
      diff(f, coords[0]) - diff(f, coords[1]) 
  ])

In [ ]:
gradient(sympy_model, base)

In [ ]:
divergence(sympy_model, base)

In [ ]:
curl(sympy_model, base)

In [ ]:
# A bite more complex ! Let's find the expression of g but as a function of X0 not (X0, X1)
sp_model_f = model_f.sympy()
sp_model_g = model_g.sympy()

print("Function f           : ", sp_model_f)
print("Function g           : ", sp_model_g)

# Creation of the complete base (inputs + intermediate variables)
base = x0, x1, x2, x3 = symbols("x0 x1 x2 x3")

# Substitution
sp_model_g = sp_model_g.subs({x3 : sp_model_f})

# Final result
print("Function g (complete): \n\n")
pprint(sp_model_g, use_unicode = True)